In [1]:
import import_ipynb

In [2]:
%run requirements.ipynb
%run config.ipynb
%run module_preprocess.ipynb
%run module_feature_extraction.ipynb
%run module_training.ipynb

In [3]:
four_time = pd.read_csv("CSV/four_time.csv")
v_value = pd.read_csv("CSV/v_value.csv")
mic = pd.read_csv("CSV/mic.csv")
dwt_distance_eu = pd.read_csv("CSV/tsc_dtw_eu_10_48.csv", names=['meter_dtw_eu', 'meter_sdtw_eu'], header=0)
dwt_distance_sdtw = pd.read_csv("CSV/tsc_dtw_sdtw_10_48.csv", names=['meter_dtw_sdtw', 'meter_sdtw_sdtw'], header=0)
# smart_meter = pd.read_csv("CSV/smart_meter.csv")
# fdi = pd.read_csv("CSV/FDI.csv")
print(four_time.shape, v_value.shape, mic.shape, dwt_distance_eu.shape, dwt_distance_sdtw.shape)

(2239250, 7) (2239250, 9) (2239250, 4) (2239250, 2) (2239250, 2)


In [4]:
dwt_distance_eu.columns, dwt_distance_sdtw.columns

(Index(['meter_dtw_eu', 'meter_sdtw_eu'], dtype='object'),
 Index(['meter_dtw_sdtw', 'meter_sdtw_sdtw'], dtype='object'))

In [5]:
dwt_distance_sdtw.head(), dwt_distance_sdtw.dtypes

(   meter_dtw_sdtw  meter_sdtw_sdtw
 0        1.983855       -69.851348
 1        2.297941       -70.646866
 2        2.330373       -69.688795
 3        2.382700       -70.707523
 4        2.022757       -71.856831,
 meter_dtw_sdtw     float64
 meter_sdtw_sdtw    float64
 dtype: object)

## prepare data

In [6]:
training_data = concat_two_df_and_drop_same_column_in_df2(four_time, v_value)
training_data = concat_two_df_and_drop_same_column_in_df2(training_data, mic)
training_data = concat_two_df_and_drop_same_column_in_df2(training_data, dwt_distance_eu)
training_data = concat_two_df_and_drop_same_column_in_df2(training_data, dwt_distance_sdtw)

,MeterID,Day,FDI,overnight,breakfast,daytime,evening
0,1002,2009-07-15,False,0.152176,0.08725,0.151714,0.099615


,MeterID,Day,FDI,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1
0,1002,2009-07-15,False,19,46,37,0.026737,0.02806,0.000975


,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1
0,1002,2009-07-15,False,0.152176,0.08725,0.151714,0.099615,19,46,37,0.026737,0.02806,0.000975


,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1
0,1002,2009-07-15,False,0.152176,0.08725,0.151714,0.099615,19,46,37,0.026737,0.02806,0.000975


,MeterID,Day,FDI,mic
0,1002,2009-07-15,False,0.604958


...

,meter_dtw_eu,meter_sdtw_eu
0,1.656341,-66.713528


,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_eu,meter_sdtw_eu
0,1002,2009-07-15,False,0.152176,0.08725,0.151714,0.099615,19,46,37,0.026737,0.02806,0.000975,0.604958,1.656341,-66.713528


,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_eu,meter_sdtw_eu
0,1002,2009-07-15,False,0.152176,0.08725,0.151714,0.099615,19,46,37,0.026737,0.02806,0.000975,0.604958,1.656341,-66.713528


,meter_dtw_sdtw,meter_sdtw_sdtw
0,1.983855,-69.851348


,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_eu,meter_sdtw_eu,meter_dtw_sdtw,meter_sdtw_sdtw
0,1002,2009-07-15,False,0.152176,0.08725,0.151714,0.099615,19,46,37,0.026737,0.02806,0.000975,0.604958,1.656341,-66.713528,1.983855,-69.851348


In [7]:
training_data.head(3)

,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_eu,meter_sdtw_eu,meter_dtw_sdtw,meter_sdtw_sdtw
0,1002,2009-07-15,False,0.152176,0.08725,0.151714,0.099615,19,46,37,0.026737,0.028060,0.000975,0.604958,1.656341,-66.713528,1.983855,-69.851348
1,1002,2009-07-16,False,0.154059,0.06725,0.293143,0.209538,19,35,21,0.065311,0.021794,0.007450,0.722518,1.894607,-68.912719,2.297941,-70.646866
2,1002,2009-07-17,False,0.119647,0.09575,0.304214,0.162692,20,30,26,0.066147,0.033839,0.006058,0.662717,1.915961,-67.859950,2.330373,-69.688795


In [8]:
smart_meter_data_group = []
for i in range(10):
    d1 = pd.read_csv('CSV/smartmeterhalf/smart_meter_'+str(i)+'_1.csv')
    d2 = pd.read_csv('CSV/smartmeterhalf/smart_meter_'+str(i)+'_2.csv')
    data = pd.concat([d1,d2])
    print(data.shape)
    smart_meter_data_group.append(data)
    
smart_meter = []
for area in smart_meter_data_group:
    t = area['Atk_Elec'].to_numpy().reshape(-1, 530, 48)
    for meter in t:
        smart_meter.append(meter)        
smart_meter = np.asarray(smart_meter)
print(smart_meter.shape)
smart_meter = smart_meter.reshape(-1, 48)
smart_meter_df = pd.DataFrame(smart_meter)
print(smart_meter_df.shape)

(10735680, 8)
(10735680, 8)
(10735680, 8)
(10735680, 8)
(10735680, 8)
(10735680, 8)
(10735680, 8)
(10735680, 8)
(10735680, 8)
(10862880, 8)
(4225, 530, 48)
(2239250, 48)


In [9]:
training_data = concat_two_df_and_drop_same_column_in_df2(training_data, smart_meter_df)

,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_eu,meter_sdtw_eu,meter_dtw_sdtw,meter_sdtw_sdtw
0,1002,2009-07-15,False,0.152176,0.08725,0.151714,0.099615,19,46,37,0.026737,0.02806,0.000975,0.604958,1.656341,-66.713528,1.983855,-69.851348


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,0.362,0.064,0.119,0.023,0.14,0.036,0.108,0.083,0.056,0.129,0.015,0.132,0.054,0.082,0.103,0.028,0.136,0.051,0.333,0.384,0.066,0.138,0.077,0.063,0.15,0.084,0.202,0.108,0.239,0.178,0.051,0.135,0.017,0.116,0.063,0.068,0.118,0.014,0.132,0.053,0.078,0.126,0.128,0.247,0.249,0.305,0.403,0.309


,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_eu,meter_sdtw_eu,meter_dtw_sdtw,meter_sdtw_sdtw,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,1002,2009-07-15,False,0.152176,0.08725,0.151714,0.099615,19,46,37,0.026737,0.02806,0.000975,0.604958,1.656341,-66.713528,1.983855,-69.851348,0.362,0.064,0.119,0.023,0.14,0.036,0.108,0.083,0.056,0.129,0.015,0.132,0.054,0.082,0.103,0.028,0.136,0.051,0.333,0.384,0.066,0.138,0.077,0.063,0.15,0.084,0.202,0.108,0.239,0.178,0.051,0.135,0.017,0.116,0.063,0.068,0.118,0.014,0.132,0.053,0.078,0.126,0.128,0.247,0.249,0.305,0.403,0.309


In [10]:
training_data.columns

Index(['MeterID', 'Day', 'FDI', 'overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_eu', 'meter_sdtw_eu', 'meter_dtw_sdtw', 'meter_sdtw_sdtw', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47], dtype='object')

In [11]:
Counter(training_data['FDI'])

Counter({False: 2127950, True: 111300})

## classifier

In [12]:
xgboost = XGBClassifier(scale_pos_weight=1, use_label_encoder=False, n_jobs=6, verbosity=0)
adaboost = AdaBoostClassifier()
lineSVM = LinearSVC(max_iter=100, C=50) # kernel = rbf
randforest = RandomForestClassifier(n_jobs=6)
complementNB = ComplementNB()
DT = DecisionTreeClassifier()
kNN = KNeighborsClassifier(n_neighbors=2, n_jobs=1)

In [13]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_eu', 'meter_sdtw_eu', 'meter_dtw_sdtw', 'meter_sdtw_sdtw']
y_col = ['FDI']
training_sample = training_data.iloc[:422*3*530]

In [14]:
training_data.columns

Index(['MeterID', 'Day', 'FDI', 'overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_eu', 'meter_sdtw_eu', 'meter_dtw_sdtw', 'meter_sdtw_sdtw', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47], dtype='object')

## XGBoost

#### xgboost all features include origin data

In [32]:
training_data.shape

(2239250, 66)

In [37]:
training_data.head()

,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_eu,meter_sdtw_eu,meter_dtw_sdtw,meter_sdtw_sdtw,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,1002,2009-07-15,False,0.152176,0.08725,0.151714,0.099615,19,46,37,0.026737,0.028060,0.000975,0.604958,1.656341,-66.713528,1.983855,-69.851348,0.362,0.064,0.119,0.023,0.140,0.036,0.108,0.083,0.056,0.129,0.015,0.132,0.054,0.082,0.103,0.028,0.136,0.051,0.333,0.384,0.066,0.138,0.077,0.063,0.150,0.084,0.202,0.108,0.239,0.178,0.051,0.135,0.017,0.116,0.063,0.068,0.118,0.014,0.132,0.053,0.078,0.126,0.128,0.247,0.249,0.305,0.403,0.309
1,1002,2009-07-16,False,0.154059,0.06725,0.293143,0.209538,19,35,21,0.065311,0.021794,0.007450,0.722518,1.894607,-68.912719,2.297941,-70.646866,0.409,0.072,0.075,0.109,0.016,0.141,0.030,0.103,0.074,0.060,0.130,0.015,0.119,0.067,0.057,0.130,0.015,0.209,0.434,0.938,0.264,0.107,0.199,0.145,0.304,0.223,0.287,0.294,0.198,0.305,0.197,0.287,0.252,0.236,0.233,0.313,0.153,0.138,0.191,0.091,0.215,0.182,0.207,0.226,0.092,0.184,0.067,0.179
2,1002,2009-07-17,False,0.119647,0.09575,0.304214,0.162692,20,30,26,0.066147,0.033839,0.006058,0.662717,1.915961,-67.859950,2.330373,-69.688795,0.283,0.343,0.080,0.067,0.130,0.017,0.139,0.052,0.094,0.099,0.042,0.141,0.025,0.115,0.076,0.061,0.131,0.067,0.300,0.512,0.950,0.636,0.315,0.105,0.110,0.201,0.087,0.201,0.127,0.162,0.486,0.096,0.214,0.110,0.178,0.152,0.138,0.121,0.096,0.208,0.098,0.170,0.258,0.276,0.171,0.074,0.197,0.083
3,1002,2009-07-18,False,0.134588,0.21375,0.194429,0.189615,16,37,23,0.040524,0.024648,0.002019,0.631590,2.070341,-68.254153,2.382700,-70.707523,0.306,0.461,0.144,0.140,0.031,0.116,0.074,0.071,0.119,0.019,0.143,0.046,0.093,0.096,0.038,0.139,0.582,0.384,0.260,0.239,0.275,0.203,0.309,0.029,0.154,0.124,0.167,0.141,0.142,0.180,0.115,0.155,0.348,0.216,0.101,0.206,0.128,0.354,0.179,0.107,0.209,0.107,0.187,0.168,0.117,0.142,0.076,0.166
4,1002,2009-07-19,False,0.166353,0.09425,0.215357,0.222154,21,30,24,0.042195,0.042334,0.006197,0.830381,1.556185,-70.266595,2.022757,-71.856831,0.054,0.464,0.488,0.406,0.307,0.024,0.143,0.041,0.107,0.086,0.059,0.132,0.016,0.130,0.059,0.081,0.107,0.025,0.140,0.038,0.094,0.606,0.490,0.165,0.089,0.212,0.122,0.154,0.173,0.099,0.608,0.275,0.245,0.215,0.267,0.174,0.291,0.139,0.187,0.143,0.154,0.273,0.259,0.266,0.362,0.399,0.327,0.424


In [38]:
# only origin 
# no dtw sdtw
x_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 3 minutes and 22.83 seconds.

 Time taken: 0 hours 3 minutes and 18.37 seconds.

 Time taken: 0 hours 3 minutes and 6.85 seconds.
average_precision_score: 0.9291145309002503
average_recall_score: 0.5056903264450434
average_accuracy_score: 0.9735134531651223
average_roc_auc_score: 0.7518363675897922
average_f1_score: 0.6549115339164248


In [39]:
# origin and featrue with no dtw sdtw
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_eu', 'meter_sdtw_eu', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 3 minutes and 58.29 seconds.

 Time taken: 0 hours 4 minutes and 18.14 seconds.

 Time taken: 0 hours 4 minutes and 15.12 seconds.
average_precision_score: 0.9441207873161588
average_recall_score: 0.5662274133972248
average_accuracy_score: 0.9767739694590203
average_roc_auc_score: 0.7822372763313575
average_f1_score: 0.7078874928713406


In [46]:
# origin and featrue with no no dtw sdtw and cmax
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'mic', 'meter_dtw_eu', 'meter_sdtw_eu', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 5 minutes and 16.51 seconds.

 Time taken: 0 hours 4 minutes and 30.86 seconds.

 Time taken: 0 hours 5 minutes and 25.26 seconds.
average_precision_score: 0.9390081674378257
average_recall_score: 0.5467006089647599
average_accuracy_score: 0.9757041668217284
average_roc_auc_score: 0.7724216590202633
average_f1_score: 0.6910586688050041


In [55]:
# origin and four time
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 4 minutes and 36.93 seconds.

 Time taken: 0 hours 5 minutes and 2.61 seconds.

 Time taken: 0 hours 5 minutes and 42.03 seconds.
average_precision_score: 0.9339160323460906
average_recall_score: 0.5058700209643606
average_accuracy_score: 0.9736603277387023
average_roc_auc_score: 0.7519987938313087
average_f1_score: 0.6562395327222353


In [48]:
# origin and vvalue
x_col = ['Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 3 minutes and 55.55 seconds.

 Time taken: 0 hours 3 minutes and 45.51 seconds.

 Time taken: 0 hours 5 minutes and 4.3 seconds.
average_precision_score: 0.9382426717189306
average_recall_score: 0.5574722970949386
average_accuracy_score: 0.9761810129879797
average_roc_auc_score: 0.7777766961793842
average_f1_score: 0.6993644517816743


In [54]:
# origin and mic
x_col = ['mic', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 3 minutes and 13.96 seconds.

 Time taken: 0 hours 3 minutes and 59.22 seconds.

 Time taken: 0 hours 4 minutes and 28.52 seconds.
average_precision_score: 0.9279665661222798
average_recall_score: 0.5062893081761006
average_accuracy_score: 0.9735074987905176
average_roc_auc_score: 0.7521170610211705
average_f1_score: 0.6551272725439258


In [56]:
# origin and featrue with eu
x_col = ['meter_dtw_eu', 'meter_sdtw_eu', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 4 minutes and 36.28 seconds.

 Time taken: 0 hours 4 minutes and 50.0 seconds.

 Time taken: 0 hours 4 minutes and 36.29 seconds.
average_precision_score: 0.9273053107855099
average_recall_score: 0.5550863531995608
average_accuracy_score: 0.9757230223413097
average_roc_auc_score: 0.7764051486072682
average_f1_score: 0.6944635296781025


In [51]:
# origin and featrue with sdtw
x_col = ['meter_dtw_sdtw', 'meter_sdtw_sdtw', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 3 minutes and 56.88 seconds.

 Time taken: 0 hours 4 minutes and 22.61 seconds.

 Time taken: 0 hours 4 minutes and 2.36 seconds.
average_precision_score: 0.9295543105166543
average_recall_score: 0.5424478386742538
average_accuracy_score: 0.9752144195105008
average_roc_auc_score: 0.7701488105339229
average_f1_score: 0.6850999755563145


In [53]:
# origin and all featrue
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_eu', 'meter_sdtw_eu', 'meter_dtw_sdtw', 'meter_sdtw_sdtw', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 6 minutes and 15.41 seconds.

 Time taken: 0 hours 5 minutes and 0.08 seconds.

 Time taken: 0 hours 6 minutes and 21.95 seconds.
average_precision_score: 0.9436379874722004
average_recall_score: 0.5666367175801138
average_accuracy_score: 0.97677793904209
average_roc_auc_score: 0.7824333129321498
average_f1_score: 0.708077525710987


#### xgboos all features

In [57]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_eu', 'meter_sdtw_eu', 'meter_dtw_sdtw', 'meter_sdtw_sdtw']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 2 minutes and 16.39 seconds.

 Time taken: 0 hours 1 minutes and 57.59 seconds.

 Time taken: 0 hours 2 minutes and 13.72 seconds.
average_precision_score: 0.8872081384306183
average_recall_score: 0.4140561046221423
average_accuracy_score: 0.9682607023681044
average_roc_auc_score: 0.7056519234859892
average_f1_score: 0.5645977640275887


#### xgboost no sdtw

In [18]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_eu', 'meter_sdtw_eu']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 1 minutes and 36.28 seconds.

 Time taken: 0 hours 1 minutes and 43.86 seconds.

 Time taken: 0 hours 1 minutes and 43.92 seconds.
average_precision_score: 0.3791271791417423
average_recall_score: 0.6039632624538285
average_accuracy_score: 0.9311465892597969
average_roc_auc_score: 0.7761114014021743
average_f1_score: 0.4658252095703581


#### xgboost no eu

In [13]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_sdtw', 'meter_sdtw_sdtw']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 1 minutes and 43.72 seconds.

 Time taken: 0 hours 1 minutes and 41.23 seconds.

 Time taken: 0 hours 1 minutes and 44.22 seconds.
average_precision_score: 0.8848252302915997
average_recall_score: 0.40885494659079563
average_accuracy_score: 0.9679724113976654
average_roc_auc_score: 0.7030356799418573
average_f1_score: 0.5592770631726028


#### xgboost whole weight=10

In [19]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_eu', 'meter_sdtw_eu', 'meter_dtw_sdtw', 'meter_sdtw_sdtw']
xgboost = XGBClassifier(scale_pos_weight=10, use_label_encoder=False, n_jobs=-1, verbosity=0)
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)
xgboost = XGBClassifier(scale_pos_weight=1, use_label_encoder=False, n_jobs=6, verbosity=0)


 Time taken: 0 hours 1 minutes and 49.77 seconds.

 Time taken: 0 hours 1 minutes and 49.48 seconds.

 Time taken: 0 hours 1 minutes and 51.38 seconds.
average_precision_score: 0.3739503779486825
average_recall_score: 0.604702006588799
average_accuracy_score: 0.9300291516256683
average_roc_auc_score: 0.7758735119164171
average_f1_score: 0.46211462397773256


#### four_time

In [20]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 0 minutes and 37.69 seconds.

 Time taken: 0 hours 0 minutes and 38.13 seconds.

 Time taken: 0 hours 0 minutes and 37.13 seconds.
average_precision_score: 0.8501484851517773
average_recall_score: 0.24338624338624337
average_accuracy_score: 0.9602595114931836
average_roc_auc_score: 0.6205704971535936
average_f1_score: 0.37839530222854306


#### v_value

In [21]:
x_col = [ 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 0 minutes and 43.59 seconds.

 Time taken: 0 hours 0 minutes and 41.73 seconds.

 Time taken: 0 hours 0 minutes and 40.82 seconds.
average_precision_score: 0.8338801465777607
average_recall_score: 0.3675351901766996
average_accuracy_score: 0.9649257563916489
average_roc_auc_score: 0.6818533897107172
average_f1_score: 0.5101863882751242


#### mic

In [22]:
x_col = ['mic']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 0 minutes and 27.82 seconds.


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 Time taken: 0 hours 0 minutes and 28.69 seconds.


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 Time taken: 0 hours 0 minutes and 28.47 seconds.


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


average_precision_score: 0.0
average_recall_score: 0.0
average_accuracy_score: 0.9502958579881656
average_roc_auc_score: 0.5
average_f1_score: 0.0


#### tsc_eu

In [23]:
x_col = ['meter_dtw_eu', 'meter_sdtw_eu']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 1 minutes and 2.3 seconds.

 Time taken: 0 hours 0 minutes and 59.77 seconds.

 Time taken: 0 hours 1 minutes and 2.49 seconds.
average_precision_score: 0.5603285403008917
average_recall_score: 0.012478786063691724
average_accuracy_score: 0.9504288390210016
average_roc_auc_score: 0.5059830169160745
average_f1_score: 0.024409222723381422


#### tsc_sddtw

In [24]:
x_col = ['meter_dtw_sdtw', 'meter_sdtw_sdtw']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 1 minutes and 1.17 seconds.

 Time taken: 0 hours 1 minutes and 1.23 seconds.

 Time taken: 0 hours 1 minutes and 0.14 seconds.
average_precision_score: 0.7888226114970301
average_recall_score: 0.0011280822601577318
average_accuracy_score: 0.9503370424125142
average_roc_auc_score: 0.5005562088658496
average_f1_score: 0.002252893095257771


#### tsc eu & sdtw

In [25]:
x_col = ['meter_dtw_eu', 'meter_sdtw_eu', 'meter_dtw_sdtw', 'meter_sdtw_sdtw']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 1 minutes and 10.67 seconds.

 Time taken: 0 hours 1 minutes and 8.76 seconds.

 Time taken: 0 hours 1 minutes and 9.62 seconds.
average_precision_score: 0.6933222818165969
average_recall_score: 0.07812718378756113
average_accuracy_score: 0.9524597769590514
average_roc_auc_score: 0.5381589653753004
average_f1_score: 0.14038157426537531


In [130]:
# old data
x_col = ['meter_dtw_eu', 'meter_sdtw_eu', 'meter_dtw_sdtw', 'meter_sdtw_sdtw']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 3 minutes and 19.53 seconds.
average_precision_score: 0.7030109489051095
average_recall_score: 0.0346136567834681
average_accuracy_score: 0.9024635607618707
average_roc_auc_score: 0.5164987350451601
average_f1_score: 0.06597876348689845


## model evaluate unbalanced (daily & 4 methods)

In [19]:
# origin and all featrue
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_eu', 'meter_sdtw_eu', 'meter_dtw_sdtw', 'meter_sdtw_sdtw', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 4 minutes and 59.63 seconds.

 Time taken: 0 hours 5 minutes and 21.49 seconds.

 Time taken: 0 hours 5 minutes and 40.25 seconds.
average_precision_score: 0.9436379874722004
average_recall_score: 0.5666367175801138
average_accuracy_score: 0.97677793904209
average_roc_auc_score: 0.7824333129321498
average_f1_score: 0.708077525710987


In [20]:
print("xgboost: ")
simple_train(xgboost, training_data, x_col, y_col)
print("adaboost: ")
simple_train(adaboost, training_data, x_col, y_col)
print("rf: ")
simple_train(randforest, training_data, x_col, y_col)

xgboost: 
(1791400, 66)
(447850, 66)

training time:  Time taken: 0 hours 5 minutes and 42.57 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.45 seconds.
accuracy_score: 0.9754203416322429
precision score: 0.9031818833309445
recall_score: 0.5661725067385445
roc_auc_score: 0.7814990450232114
f1_score: 0.6960291599933728
adaboost: 
(1791400, 66)
(447850, 66)

training time:  Time taken: 0 hours 11 minutes and 14.54 seconds.

predict time:  Time taken: 0 hours 0 minutes and 6.06 seconds.
accuracy_score: 0.9581132075471698
precision score: 0.7425523070527921
recall_score: 0.2407457322551662
roc_auc_score: 0.6181900140868867
f1_score: 0.3636055229500967
rf: 
(1791400, 66)
(447850, 66)

training time:  Time taken: 0 hours 11 minutes and 1.48 seconds.

predict time:  Time taken: 0 hours 0 minutes and 5.16 seconds.
accuracy_score: 0.9722719660600647
precision score: 0.9421383647798742
recall_score: 0.4710691823899371
roc_auc_score: 0.7347779944704216
f1_score: 0.6280922431865829
n

ValueError: Negative values in data passed to ComplementNB (input X)

In [26]:
training_data.head(1)

,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_eu,meter_sdtw_eu,meter_dtw_sdtw,meter_sdtw_sdtw,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,1002,2009-07-15,False,0.152176,0.08725,0.151714,0.099615,19,46,37,0.026737,0.02806,0.000975,0.604958,1.656341,-66.713528,1.983855,-69.851348,0.362,0.064,0.119,0.023,0.14,0.036,0.108,0.083,0.056,0.129,0.015,0.132,0.054,0.082,0.103,0.028,0.136,0.051,0.333,0.384,0.066,0.138,0.077,0.063,0.15,0.084,0.202,0.108,0.239,0.178,0.051,0.135,0.017,0.116,0.063,0.068,0.118,0.014,0.132,0.053,0.078,0.126,0.128,0.247,0.249,0.305,0.403,0.309


In [30]:
print("nb: ")
simple_train(complementNB, training_data, x_col, y_col, minmax=True)
print("dt: ")
simple_train(DT, training_data, x_col, y_col)
print("knn: ")
simple_train(kNN, training_data, x_col, y_col)

nb: 
(1791400, 66)
(447850, 66)

training time:  Time taken: 0 hours 0 minutes and 0.63 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.09 seconds.
accuracy_score: 0.6785821145472815
precision score: 0.09502732276808594
recall_score: 0.6413746630727762
roc_auc_score: 0.660951435486199
f1_score: 0.16552947519144817
dt: 
(1791400, 66)
(447850, 66)

training time:  Time taken: 0 hours 8 minutes and 22.05 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.43 seconds.
accuracy_score: 0.9366528971753936
precision score: 0.39321168903803133
recall_score: 0.5053459119496856
roc_auc_score: 0.7322789147614683
f1_score: 0.44228198474482977
knn: 
(1791400, 66)
(447850, 66)

training time:  Time taken: 0 hours 0 minutes and 0.46 seconds.


KeyboardInterrupt: 

## SMOTE

In [17]:
training_data.columns

Index(['MeterID', 'Day', 'FDI', 'overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_eu', 'meter_sdtw_eu', 'meter_dtw_sdtw', 'meter_sdtw_sdtw', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47], dtype='object')

In [37]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_eu', 'meter_sdtw_eu', 'meter_dtw_sdtw', 'meter_sdtw_sdtw', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]

In [38]:
oversample = SMOTE()
oversample_X, oversample_y = oversample.fit_resample(training_data[x_col], training_data['FDI'])

In [39]:
Counter(oversample_y)

Counter({False: 2127950, True: 2127950})

In [40]:
oversample_y = pd.DataFrame(oversample_y, columns=['FDI'])

In [41]:
smote_data = concat_two_df_and_drop_same_column_in_df2(oversample_X, oversample_y)

,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_eu,meter_sdtw_eu,meter_dtw_sdtw,meter_sdtw_sdtw,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,0.152176,0.08725,0.151714,0.099615,19,46,37,0.026737,0.02806,0.000975,0.604958,1.656341,-66.713528,1.983855,-69.851348,0.362,0.064,0.119,0.023,0.14,0.036,0.108,0.083,0.056,0.129,0.015,0.132,0.054,0.082,0.103,0.028,0.136,0.051,0.333,0.384,0.066,0.138,0.077,0.063,0.15,0.084,0.202,0.108,0.239,0.178,0.051,0.135,0.017,0.116,0.063,0.068,0.118,0.014,0.132,0.053,0.078,0.126,0.128,0.247,0.249,0.305,0.403,0.309


,FDI
0,False


,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_eu,meter_sdtw_eu,meter_dtw_sdtw,meter_sdtw_sdtw,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,FDI
0,0.152176,0.08725,0.151714,0.099615,19,46,37,0.026737,0.02806,0.000975,0.604958,1.656341,-66.713528,1.983855,-69.851348,0.362,0.064,0.119,0.023,0.14,0.036,0.108,0.083,0.056,0.129,0.015,0.132,0.054,0.082,0.103,0.028,0.136,0.051,0.333,0.384,0.066,0.138,0.077,0.063,0.15,0.084,0.202,0.108,0.239,0.178,0.051,0.135,0.017,0.116,0.063,0.068,0.118,0.014,0.132,0.053,0.078,0.126,0.128,0.247,0.249,0.305,0.403,0.309,False


In [ ]:
# smote on training_sample
model, scoe = train_single_model(xgboost, smote_data, x_col, y_col, split_num = 1, return_model=True)

In [182]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_48', 'meter_sdtw_48', 'meter_dtw_530', 'meter_sdtw_530']
# smote on all data
model, scoe = train_single_model(xgboost, smote_data, x_col, y_col, split_num = 3, return_model=True)

# imbalanced data 
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 6 minutes and 56.95 seconds.

 Time taken: 0 hours 7 minutes and 1.75 seconds.

 Time taken: 0 hours 6 minutes and 54.53 seconds.
average_precision_score: 0.8459640082767406
average_recall_score: 0.7223315895172687
average_accuracy_score: 0.7954040942487127
average_roc_auc_score: 0.7954040942487128
average_f1_score: 0.7792693628677908

 Time taken: 0 hours 3 minutes and 25.23 seconds.

 Time taken: 0 hours 2 minutes and 58.47 seconds.

 Time taken: 0 hours 3 minutes and 24.26 seconds.
average_precision_score: 0.9560228141206603
average_recall_score: 0.24511829889188377
average_accuracy_score: 0.9238375944326598
average_roc_auc_score: 0.6219368302532212
average_f1_score: 0.39018849693571084


## model evaluate smote ( daily & four features)

In [15]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_eu', 'meter_sdtw_eu', 'meter_dtw_sdtw', 'meter_sdtw_sdtw', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
training_num = int(training_data.shape[0] * 0.8)
train = training_data.iloc[:training_num]
test = training_data.iloc[training_num:]
print(Counter(train['FDI']))
oversample = SMOTE()
oversample_X, oversample_y = oversample.fit_resample(train[x_col], train['FDI'])
print(Counter(oversample_y))
oversample_y = pd.DataFrame(oversample_y, columns=['FDI'])
smote_train_data = concat_two_df_and_drop_same_column_in_df2(oversample_X, oversample_y)

Counter({False: 1702360, True: 89040})
Counter({False: 1702360, True: 1702360})


,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_eu,meter_sdtw_eu,meter_dtw_sdtw,meter_sdtw_sdtw,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,0.152176,0.08725,0.151714,0.099615,19,46,37,0.026737,0.02806,0.000975,0.604958,1.656341,-66.713528,1.983855,-69.851348,0.362,0.064,0.119,0.023,0.14,0.036,0.108,0.083,0.056,0.129,0.015,0.132,0.054,0.082,0.103,0.028,0.136,0.051,0.333,0.384,0.066,0.138,0.077,0.063,0.15,0.084,0.202,0.108,0.239,0.178,0.051,0.135,0.017,0.116,0.063,0.068,0.118,0.014,0.132,0.053,0.078,0.126,0.128,0.247,0.249,0.305,0.403,0.309


,FDI
0,False


,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_eu,meter_sdtw_eu,meter_dtw_sdtw,meter_sdtw_sdtw,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,FDI
0,0.152176,0.08725,0.151714,0.099615,19,46,37,0.026737,0.02806,0.000975,0.604958,1.656341,-66.713528,1.983855,-69.851348,0.362,0.064,0.119,0.023,0.14,0.036,0.108,0.083,0.056,0.129,0.015,0.132,0.054,0.082,0.103,0.028,0.136,0.051,0.333,0.384,0.066,0.138,0.077,0.063,0.15,0.084,0.202,0.108,0.239,0.178,0.051,0.135,0.017,0.116,0.063,0.068,0.118,0.014,0.132,0.053,0.078,0.126,0.128,0.247,0.249,0.305,0.403,0.309,False


In [16]:
# tune xgboost
xgboost = XGBClassifier(scale_pos_weight=1, use_label_encoder=False, verbosity=0, max_depth=8, subsample=0.8)
smote_train(xgboost, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 19 minutes and 41.8 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.5 seconds.
accuracy_score: 0.9416210784861002
precision score: 0.44034274131629864
recall_score: 0.644115004492363
roc_auc_score: 0.8006483995886943
f1_score: 0.523084219550902


In [16]:
# tune xgboost 2
xgboost = XGBClassifier(scale_pos_weight=1, use_label_encoder=False, verbosity=0, max_depth=9, subsample=0.8)
smote_train(xgboost, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 23 minutes and 7.89 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.54 seconds.
accuracy_score: 0.9483510103829407
precision score: 0.4855837664272237
recall_score: 0.658984725965858
roc_auc_score: 0.8112353550645098
f1_score: 0.5591492119156073


In [18]:
# tune xgboost 3
xgboost = XGBClassifier(scale_pos_weight=1, use_label_encoder=False, verbosity=0, max_depth=10, subsample=0.8)
smote_train(xgboost, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 28 minutes and 1.27 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.6 seconds.
accuracy_score: 0.9550206542369096
precision score: 0.5393220842934662
recall_score: 0.6518867924528302
roc_auc_score: 0.8113812589581523
f1_score: 0.5902859699792539


In [19]:
# tune xgboost 4
xgboost = XGBClassifier(scale_pos_weight=1, use_label_encoder=False, verbosity=0, max_depth=10)
smote_train(xgboost, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 29 minutes and 5.47 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.6 seconds.
accuracy_score: 0.9548911465892598
precision score: 0.5389713679745494
recall_score: 0.639308176100629
roc_auc_score: 0.8053527651808862
f1_score: 0.5848676639815881


In [20]:
# tune xgboost 5
xgboost = XGBClassifier(scale_pos_weight=1, use_label_encoder=False, verbosity=0, max_depth=10, subsample=0.9)
smote_train(xgboost, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 26 minutes and 35.75 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.59 seconds.
accuracy_score: 0.9554270403036731
precision score: 0.5437614259597806
recall_score: 0.6413746630727762
roc_auc_score: 0.8066139275560315
f1_score: 0.5885481078407123


In [21]:
# tune xgboost 6
xgboost = XGBClassifier(scale_pos_weight=1, use_label_encoder=False, verbosity=0, max_depth=10, subsample=0.7)
smote_train(xgboost, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 25 minutes and 42.98 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.59 seconds.
accuracy_score: 0.9538751814223513
precision score: 0.5295789201756652
recall_score: 0.6446540880503144
roc_auc_score: 0.8073513632055891
f1_score: 0.5814778045667282


In [22]:
# tune xgboost ˙
xgboost = XGBClassifier(scale_pos_weight=1, use_label_encoder=False, verbosity=0, max_depth=10, subsample=0.8, eta=0.28)
smote_train(xgboost, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 26 minutes and 47.73 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.59 seconds.
accuracy_score: 0.9543373897510327
precision score: 0.5339485332733138
recall_score: 0.6394429469901168
roc_auc_score: 0.8051252658773982
f1_score: 0.5819534731591643


In [50]:
smote_train(xgboost, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 21 minutes and 54.41 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.46 seconds.
accuracy_score: 0.9153332589036508
precision score: 0.33001867210907987
recall_score: 0.6828391734052112
roc_auc_score: 0.8051663852646018
f1_score: 0.4449778974794345


In [51]:
smote_train(adaboost, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 39 minutes and 38.59 seconds.

predict time:  Time taken: 0 hours 0 minutes and 6.76 seconds.
accuracy_score: 0.7686569163782516
precision score: 0.14244648586875303
recall_score: 0.7279424977538185
roc_auc_score: 0.7493644677025395
f1_score: 0.23826783810609123


In [52]:
smote_train(randforest, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 31 minutes and 5.34 seconds.

predict time:  Time taken: 0 hours 0 minutes and 6.89 seconds.
accuracy_score: 0.9549268728368874
precision score: 0.5420416768020757
recall_score: 0.60062893081761
roc_auc_score: 0.7870434768987367
f1_score: 0.5698333546434812


In [54]:
smote_train(complementNB, smote_train_data, test, x_col, y_col, minmax=True)


training time:  Time taken: 0 hours 0 minutes and 1.11 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.12 seconds.
accuracy_score: 0.6847270291392207
precision score: 0.09613229651261503
recall_score: 0.6358939802336029
roc_auc_score: 0.661587583175849
f1_score: 0.1670157222500811


In [55]:
smote_train(DT, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 19 minutes and 31.32 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.49 seconds.
accuracy_score: 0.8582382494138663
precision score: 0.19093526042759904
recall_score: 0.5721024258760108
roc_auc_score: 0.7226533417474229
f1_score: 0.2863148901728906


## smote only features

In [183]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_48', 'meter_sdtw_48', 'meter_dtw_530', 'meter_sdtw_530']
xgboost = XGBClassifier(scale_pos_weight=10, use_label_encoder=False, n_jobs=6, verbosity=0)
# smote on all data
model, scoe = train_single_model(xgboost, smote_data, x_col, y_col, split_num = 3, return_model=True)

# imbalanced data 
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 6 minutes and 44.42 seconds.

 Time taken: 0 hours 7 minutes and 27.35 seconds.

 Time taken: 0 hours 7 minutes and 13.06 seconds.
average_precision_score: 0.5525945823224182
average_recall_score: 0.9934269429223933
average_accuracy_score: 0.5945393488100452
average_roc_auc_score: 0.5945393488100452
average_f1_score: 0.7101589543269068

 Time taken: 0 hours 4 minutes and 14.14 seconds.

 Time taken: 0 hours 3 minutes and 38.09 seconds.

 Time taken: 0 hours 3 minutes and 31.95 seconds.
average_precision_score: 0.19184276099907524
average_recall_score: 0.6470200658879904
average_accuracy_score: 0.6939386947514669
average_roc_auc_score: 0.6730688447248089
average_f1_score: 0.2959347416254148


In [184]:
# smote on all data adaboost
model, scoe = train_single_model(adaboost, smote_data, x_col, y_col, split_num = 3, return_model=True)
# imbalanced data adaboost
model, score = train_single_model(adaboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 10 minutes and 59.17 seconds.

 Time taken: 0 hours 11 minutes and 8.46 seconds.

 Time taken: 0 hours 10 minutes and 58.65 seconds.
average_precision_score: 0.6971849991309447
average_recall_score: 0.5112014699846005
average_accuracy_score: 0.6445127645683022
average_roc_auc_score: 0.6445127645683022
average_f1_score: 0.5898345754145229

 Time taken: 0 hours 5 minutes and 22.34 seconds.

 Time taken: 0 hours 4 minutes and 14.73 seconds.

 Time taken: 0 hours 5 minutes and 4.94 seconds.
average_precision_score: 0.867681902535495
average_recall_score: 0.0678995707297594
average_accuracy_score: 0.9063123813776933
average_roc_auc_score: 0.533378430780682
average_f1_score: 0.12594264992418466


## model evaluate smote (48 hours only)

In [21]:
x_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
training_num = int(training_data.shape[0] * 0.8)
train = training_data.iloc[:training_num]
test = training_data.iloc[training_num:]
print(Counter(train['FDI']))
oversample = SMOTE()
oversample_X, oversample_y = oversample.fit_resample(train[x_col], train['FDI'])
print(Counter(oversample_y))
oversample_y = pd.DataFrame(oversample_y, columns=['FDI'])
smote_train_data = concat_two_df_and_drop_same_column_in_df2(oversample_X, oversample_y)

Counter({False: 1702360, True: 89040})
Counter({False: 1702360, True: 1702360})


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,0.362,0.064,0.119,0.023,0.14,0.036,0.108,0.083,0.056,0.129,0.015,0.132,0.054,0.082,0.103,0.028,0.136,0.051,0.333,0.384,0.066,0.138,0.077,0.063,0.15,0.084,0.202,0.108,0.239,0.178,0.051,0.135,0.017,0.116,0.063,0.068,0.118,0.014,0.132,0.053,0.078,0.126,0.128,0.247,0.249,0.305,0.403,0.309


,FDI
0,False


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,FDI
0,0.362,0.064,0.119,0.023,0.14,0.036,0.108,0.083,0.056,0.129,0.015,0.132,0.054,0.082,0.103,0.028,0.136,0.051,0.333,0.384,0.066,0.138,0.077,0.063,0.15,0.084,0.202,0.108,0.239,0.178,0.051,0.135,0.017,0.116,0.063,0.068,0.118,0.014,0.132,0.053,0.078,0.126,0.128,0.247,0.249,0.305,0.403,0.309,False


In [22]:
x_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
smote_train(xgboost, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 15 minutes and 49.9 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.34 seconds.
accuracy_score: 0.878858992966395
precision score: 0.2618682545589877
recall_score: 0.790251572327044
roc_auc_score: 0.8368725377319329
f1_score: 0.39338066752390005


In [23]:
x_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
smote_train(adaboost, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 32 minutes and 30.92 seconds.

predict time:  Time taken: 0 hours 0 minutes and 5.01 seconds.
accuracy_score: 0.7256983364965949
precision score: 0.12432024620533047
recall_score: 0.747663971248877
roc_auc_score: 0.7361067101245443
f1_score: 0.21319140214690135


In [28]:
x_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
smote_train(lineSVM, smote_train_data, test, x_col, y_col)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)



training time:  Time taken: 2 hours 49 minutes and 31.13 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.13 seconds.
accuracy_score: 0.9502958579881656


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision score: 0.0
recall_score: 0.0
roc_auc_score: 0.5
f1_score: 0.0


In [24]:
x_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
smote_train(randforest, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 25 minutes and 29.11 seconds.

predict time:  Time taken: 0 hours 0 minutes and 6.49 seconds.
accuracy_score: 0.942643742324439
precision score: 0.4461010977949734
recall_score: 0.6371069182389937
roc_auc_score: 0.797865707997525
f1_score: 0.5247636491461769


In [25]:
x_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
smote_train(complementNB, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 0 minutes and 0.86 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.15 seconds.
accuracy_score: 0.5695366752260802
precision score: 0.060355896108449825
recall_score: 0.5258310871518419
roc_auc_score: 0.5488268666803172
f1_score: 0.10828287687390432


In [26]:
x_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
smote_train(DT, smote_train_data, test, x_col, y_col)


training time:  Time taken: 0 hours 14 minutes and 21.92 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.41 seconds.
accuracy_score: 0.8221525064195602
precision score: 0.16682728592162555
recall_score: 0.6454627133872417
roc_auc_score: 0.7384283890487044
f1_score: 0.26512893850625086


## model evaluate unbalanced (48 hours only)

In [15]:
Counter(training_data['FDI'])

Counter({False: 2127950, True: 111300})

In [16]:
x_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
simple_train(xgboost, training_data, x_col, y_col)

(1791400, 66)
(447850, 66)

training time:  Time taken: 0 hours 4 minutes and 38.99 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.35 seconds.
accuracy_score: 0.9722741989505415
precision score: 0.9158428390367553
recall_score: 0.48692722371967656
roc_auc_score: 0.7422934715839861
f1_score: 0.6358117025956885


In [17]:
x_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
simple_train(adaboost, training_data, x_col, y_col)

(1791400, 66)
(447850, 66)

training time:  Time taken: 0 hours 11 minutes and 45.06 seconds.

predict time:  Time taken: 0 hours 0 minutes and 4.97 seconds.
accuracy_score: 0.9541967176509992
precision score: 0.6146475915474472
recall_score: 0.21037735849056605
roc_auc_score: 0.6017393500787143
f1_score: 0.3134643060343385


In [16]:
x_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
simple_train(randforest, training_data, x_col, y_col)

(1791400, 66)
(447850, 66)

training time:  Time taken: 0 hours 11 minutes and 21.81 seconds.

predict time:  Time taken: 0 hours 0 minutes and 4.39 seconds.
accuracy_score: 0.9713609467455622
precision score: 0.9201852841617674
recall_score: 0.4640610961365678
roc_auc_score: 0.7309778917558706
f1_score: 0.6169742578988234


In [17]:
x_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
simple_train(complementNB, training_data, x_col, y_col)

(1791400, 66)
(447850, 66)

training time:  Time taken: 0 hours 0 minutes and 0.48 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.07 seconds.
accuracy_score: 0.560160768114324
precision score: 0.06051413623100915
recall_score: 0.5403863432165319
roc_auc_score: 0.5507906950463166
f1_score: 0.10884002895403548


In [18]:
x_col = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
simple_train(DT, training_data, x_col, y_col)

(1791400, 66)
(447850, 66)

training time:  Time taken: 0 hours 5 minutes and 48.18 seconds.

predict time:  Time taken: 0 hours 0 minutes and 0.3 seconds.
accuracy_score: 0.9374857653232109
precision score: 0.3951149950638049
recall_score: 0.48544474393530995
roc_auc_score: 0.7232869998959428
f1_score: 0.43564675764478217


In [19]:
print('all finish')

all finish


In [20]:
Counter(training_data['FDI'])

Counter({False: 2127950, True: 111300})